<a href="https://colab.research.google.com/github/pikey-msc/RiesgosFinancieros/blob/master/2025-1/BondesD_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Cargar info de github

In [21]:
!rm -rf RiesgosFinancieros
!git clone "https://github.com/pikey-msc/RiesgosFinancieros/"

Cloning into 'RiesgosFinancieros'...
remote: Enumerating objects: 1100, done.
remote: Counting objects: 100% (332/332), done.
remote: Compressing objects: 100% (156/156), done.
remote: Total 1100 (delta 185), reused 279 (delta 158), pack-reused 768 (from 1)
Receiving objects: 100% (1100/1100), 63.87 MiB | 22.05 MiB/s, done.
Resolving deltas: 100% (690/690), done.
Updating files: 100% (434/434), done.


## Carga información insumos

In [22]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

fval = datetime.strptime("20230310", "%Y%m%d") #Fecha de valoración
itpl = 0 #poner 0 si se quiere interpolación lineal o 1 si se quiere tasa alambrada
alpha = 0.98 #Nivel de confianza para obtener estimaciones de riesgo

#BONDE D
# Leer datos de tasa_guber.txt
base = "RiesgosFinancieros/2024-1/Tarea/tasa_guber.txt"
btasadescst="RiesgosFinancieros/2024-1/Tarea/tasa_guber_st.txt"
btasafondeo="RiesgosFinancieros/2024-1/Tarea/tfondeo.txt"
plazos_bdm=np.array([358, 405, 550, 1200, 1800])
plazocupon_bdm=np.array([28, 28, 28, 28, 28])
contratos_bdm=np.array([220, -290, 290, -460, 100])
nominal_bdm=100

data1 = pd.read_table(base, sep="\t", header=None)
n = data1.shape[0]
m_bd = data1.shape[1]
X_orig = data1.iloc[1:n, 0:m_bd]
X1_orig = pd.DataFrame(data1.iloc[1:n, 1:m_bd], dtype=float)
X1_orig['Date'] = pd.to_datetime(X_orig[0], format='%Y%m%d')
nodos = data1.iloc[0, 1:m_bd]
n -= 1

# Leer datos de tasa_guber_st.txt
data3 = pd.read_table(btasadescst, sep="\t", header=None)
n3 = data3.shape[0]
m3_bd = data3.shape[1]
X3a_orig_bd = data3.iloc[1:n3, 0:m3_bd]
X3_orig_bd = pd.DataFrame(data3.iloc[1:n3, 1:m3_bd], dtype=float)
X3_orig_bd['Date'] = pd.to_datetime(X3a_orig_bd[0], format='%Y%m%d')
nodos3_bd = data3.iloc[0, 1:m3_bd]
n3 -= 1

# # Leer datos de tfondeo.txt
data2 = pd.read_table(btasafondeo, sep="\t", header=None)
n2 = data2.shape[0]
X2_orig = data2.iloc[1:n2, 0:2]
X2_orig_bd = X2_orig.copy()
X2_orig_bd[0] = pd.to_datetime(X2_orig_bd[0], format='%Y%m%d')
X2_orig_bd[1] = X2_orig_bd[1].astype(float)

# # Sucesión de días para tasa fondeo
tfh = pd.date_range(start=min(X2_orig_bd[0]), end=max(X2_orig_bd[0]), freq='D')
tfhd = pd.DataFrame({'fecha': tfh})


# Asegúrate de que ambos DataFrames estén ordenados por la columna de fecha de manera descendente
tfhd = tfhd.sort_values(by='fecha', ascending=True)
X2_orig_bd = X2_orig_bd.sort_values(by=0, ascending=True)  # Asumiendo que la columna 'V1' es ahora la columna 0

# Renombrar columnas para preparar para merge_asof
# tfhd = tfhd.rename(columns={'fecha': 'join_date'})
X2_orig_bd = X2_orig_bd.rename(columns={0: 'fecha'})

# Realizar el rolling join con merge_asof
X2_orig_bd = pd.merge_asof(tfhd, X2_orig_bd, on='fecha', direction='backward')


# Si quieres que el resultado esté en el DataFrame original (tfhd), puedes hacer lo siguiente:
X2_orig_bd = pd.merge(tfhd, X2_orig_bd, on='fecha', how='outer')

X1_orig = X1_orig.rename(columns={0: "fecha"})
# Filtrar tasas de fondeo

tf_act = X2_orig_bd.loc[X2_orig_bd['fecha'] == fval, 1].values[0] / 100
tf_int = X2_orig_bd.loc[(X2_orig_bd['fecha'] <= fval) & (X2_orig_bd['fecha'] >= (fval -  timedelta(int(plazocupon_bdm[0])))), 1] / 100
tf_int=tf_int[::-1]  # se hace este cambio para ordenar de mayor a menor


# Ordenar el DataFrame por la columna de unión
X1_orig = X1_orig.sort_values('Date', ascending=True)

# Agregar la columna join_date al DataFrame X1_orig
X1_orig['join_date'] = X1_orig['Date']

# Ordenar el DataFrame X2a_orig por la columna de unión
X2_orig_bd = X2_orig_bd.sort_values('fecha', ascending=True)

# Hacer el merge (join) de los DataFrames
X2_pr = pd.merge_asof(X1_orig, X2_orig_bd, left_on='Date', right_on='fecha')


## Emparejar por fecha los insumos

In [23]:
#función para unir por fecha
def filter_on_date(base_df, join_df, description):
    # Asegurarse de que ambos objetos sean DataFrames con una columna 'Date'
    if not isinstance(base_df, pd.DataFrame):
        base_df = pd.DataFrame({'Date': base_df})
    if not isinstance(join_df, pd.DataFrame):
        join_df = pd.DataFrame({'Date': join_df})

    result = pd.merge(base_df, join_df, on='Date', how='inner')[['Date']].drop_duplicates()
    print(f"Number of rows in {description}: {len(result)}")
    return result

# poner los conjuntos que se unirán quitando el primero
dataframes_to_join = [(X3_orig_bd, "Cetes_y_sobretasas"),]

#conjunto primero a unir por fecha
current_df = X1_orig

# ciclo para unir conjuntos por fecha
for join_df, description in dataframes_to_join:
    current_df = filter_on_date(current_df, join_df, description)

# Mostrar el DataFrame final
print(current_df.drop_duplicates())

# Suponemos que lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol y otras DataFrames ya están definidas

# Calcula el número de filas
n = len(current_df)

# ordenar data frame
current_df=current_df.sort_values(by="Date", ascending=False)

# Borrar las fechas ya alineadas y ordenadas y quedarse con las tasas
def join_and_drop_date(base_df, join_df, divide_by_100=False):
    result = pd.merge(base_df, join_df, on='Date', how='inner').sort_values('Date', ascending=False)
    result.drop(columns=['Date'], inplace=True)
    if divide_by_100:
        result = result / 100
    return result

# Historia de curvas de bonde
X1_orig = join_and_drop_date(current_df, X1_orig)
X1_orig.drop(columns=['join_date'], inplace=True)
X3_orig_bd = join_and_drop_date(current_df, X3_orig_bd)


Number of rows in Cetes_y_sobretasas: 255
          Date
0   2022-09-28
1   2022-09-29
2   2022-09-30
3   2022-10-03
4   2022-10-04
..         ...
250 2023-09-25
251 2023-09-26
252 2023-09-27
253 2023-09-28
254 2023-09-29

[255 rows x 1 columns]


# Enfoque a portafolio de insumos

In [25]:
m = len(plazos_bdm)
N_bd = (plazos_bdm // plazocupon_bdm) + 1  # número de cupones a pagar

# Inicialización de matrices/vectores
VTplazos_bdm = np.zeros(np.sum(N_bd))
contratos_bdmT = np.zeros(np.sum(N_bd))
plazocupon_bdmT = np.zeros(np.sum(N_bd))
tasafijaT_bd = np.zeros(np.sum(N_bd))
ulNomT_bd = np.zeros(np.sum(N_bd))
Xvp_bd = np.zeros((n, len(VTplazos_bdm)))
Xst_bd = np.zeros((n, len(VTplazos_bdm)))

# Cálculos adicionales
plazini_bd = plazos_bdm - plazocupon_bdm * (N_bd - 1)
ddv = plazocupon_bdm - plazini_bd
tfcupon = np.zeros(m)
tfcupondev = np.zeros(m)
tfcupgen = ((1 + tf_act / 360)**(plazocupon_bdm[0]) - 1) * 360 / plazocupon_bdm[0]

# Bucle para calcular tasas de cupón
for j in range(m):  # en Python los índices comienzan en 0
    tfcupondev[j] = ((np.prod(1 + tf_int[0:ddv[j]] / 360) - 1) * 360) / ddv[j]
    tfcupon[j] = (((1 + tfcupondev[j] * ddv[j] / 360) * (1 + tf_act / 360)**(plazocupon_bdm[0] - ddv[j]) - 1) * 360) / plazocupon_bdm[0]

print(tfcupondev)
print(tfcupon)

from scipy.interpolate import interp1d

for j in range(m):
    sum_N_bd = np.sum(N_bd[:j+1])
    sum_N_bd_prev = np.sum(N_bd[:j]) if j > 0 else 0

    if j == 0:
        VTplazos_bdm[:sum_N_bd] = np.arange(plazini_bd[j], plazos_bdm[j] + 1, plazocupon_bdm[j])
        contratos_bdmT[:sum_N_bd] = contratos_bdm[j]
        plazocupon_bdmT[:sum_N_bd] = plazocupon_bdm[j]
        ulNomT_bd[sum_N_bd - 1] = contratos_bdm[j]
        tasafijaT_bd[0] = tfcupon[j]
        tasafijaT_bd[1:sum_N_bd] = tfcupgen
    else:
        VTplazos_bdm[sum_N_bd_prev:sum_N_bd] = np.arange(plazini_bd[j], plazos_bdm[j] + 1, plazocupon_bdm[j])
        contratos_bdmT[sum_N_bd_prev:sum_N_bd] = contratos_bdm[j]
        plazocupon_bdmT[sum_N_bd_prev:sum_N_bd] = plazocupon_bdm[j]
        tasafijaT_bd[sum_N_bd_prev] = tfcupon[j]
        tasafijaT_bd[sum_N_bd_prev + 1:sum_N_bd] = tfcupgen
        ulNomT_bd[sum_N_bd - 1] = contratos_bdm[j]

for i in range(n):
    if itpl == 0:
        interpolator = interp1d(nodos, X1_orig.iloc[i, :], kind='linear', fill_value="interpolate")
        Xvp_bd[i, :] = interpolator(VTplazos_bdm)

        interpolator3 = interp1d(nodos3_bd, X3_orig_bd.iloc[i, :], kind='linear', fill_value="interpolate")
        Xst_bd[i, :] = interpolator3(VTplazos_bdm)
    else:
        # Código para interpolación 'talamb' (no definido en el ejemplo de R original)
        pass

X2_pr=pd.merge(X2_orig_bd, current_df, left_on='fecha', right_on="Date", how='inner').sort_values('Date', ascending=False)
X_bd_tc = (np.ones((n, len(contratos_bdmT))).T * (np.array(X2_pr[1]) / 100)).T
X_bd_ext = np.hstack((X_bd_tc, Xvp_bd, Xst_bd))


[0.11001729 0.11038289 0.11022157 0.10992531 0.11044562]
[0.10980679 0.11015967 0.10996162 0.10974557 0.11031451]


# Valuación

**Características**

Valor nominal: Generalmente, 100 pesos pero varía por emisora.

Plazo: de 1 a 360 días, según las necesidades de financiamiento de la empresa emisora.

Rendimiento: al igual que los bonos, este instrumento se compra a descuento respecto de su valor nominal, pero por lo general pagan una sobretasa referenciada a tasa fija o variable.

Garantía: este título, por ser un pagaré, no ofrece ninguna garantía, por lo que es importante evaluar bien al emisor.



$$\Large V=\sum_{i=1}^{n}\frac{N\cdot C \cdot t_{c_{p_i}}\cdot p_c/360}{(1+t_{vp_{p_i}} \cdot p_i/360)} + \frac{N\cdot C}{(1+t_{vp_{p_n}} \cdot p_n/360)}$$
	Donde: \\
$N$: Valor Nominal del bono\
$C$: Número de contratos\
$p_{c}$: Plazo fijo para cada pago de intereses del cupón.\
$p_{i}$: Plazo acumulado (en días) al cupón $i$.\
$t_{c_{p_i}}$: Tasa cupón variable, se obtiene de la curva subyacente que le corresponda, casi siempre con la tasa forward entre $p_{i-1}$ y $p_i$.\
$t_{vp_{p_i}}$: Tasa valor presente que depende de la curva de bonos según el plazo acumulado al pago del cupón $i$.


En general tenemos dos factores de riesgo subyacente (la curva de valor presente y la curva de cupones) pero como cada cupón tiene ``$n$'' flujos entonces tiene $n$ factores de riesgo para los valores presentes y $n$ factores de riesgo para los cupones, por lo que tienen $2n$ factores de riesgo específicos que provienen de dos factores de riesgo subyacentes.
Para el caso de $K$ bonos cupón variable el número de factores de riesgo sería $2\sum_{i=1}^{K}n_i$, donde $n_i$ es el número de cupones a pagar del bono $i$.

In [26]:
def bondeD(contratosT, nominal, tasafijaT, plazocuponT, VTplazos, Xvp, Xst, N, ulNomT):
    V0 = np.zeros(len(N))  # Inicializa V0, equivalente a matrix(0, 1, nrow(as.matrix(N))) en R
    V0f = (((contratosT * tasafijaT * (plazocuponT / 360)) + ulNomT) / (1 + (Xvp + Xst) * VTplazos / 360)) * nominal

    for j in range(len(N)):  # Cambia índices de 1:nrow(as.matrix(N)) a 0:(len(N)-1)
        if j == 0:  # Cambia de j==1 a j==0
            V0[j] = np.sum(V0f[:N[j]])
        else:
            V0[j] = np.sum(V0f[np.sum(N[:j]):np.sum(N[:j+1])])

    return V0

# Suponiendo que contratosT, nominal, tasafijaT, plazocuponT, VTplazos, Xvp, Xst, y N ya están definidos
V0_bd = bondeD(contratos_bdmT, nominal_bdm, tf_act, plazocupon_bdmT, VTplazos_bdm, Xvp_bd[0,], Xst_bd[0,], N_bd, ulNomT_bd)  # Cambia de Xvp[1, ] y Xst[1, ] a Xvp[0, :] y Xst[0, :]

print(V0_bd)

[ 22161.21115235 -29320.69510623  29377.07811909 -47181.61323033
  10480.40284521]
